#  Monte Carlo Tree Search
from https://medium.com/@quasimik/monte-carlo-tree-search-applied-to-letterpress-34f41c86e238



In [1]:
from datetime import datetime
import math

In [2]:
# in my game, we have an inventory 2-dim vector non-negative (representing ingrediants)
# each move is a 2-dim vector (pos and negative) for moves I can apply to my inventory to manipulate it
# 

class ItemSet(object):
    def __init__(self, items):
        self.items = items
    
    def __add__(self, addend): 
        return ItemSet(list(map(lambda x:x[0]+x[1], zip(self.items, addend.items))))
    
    def __len__(self):
        return sum(map(abs,self.items))
    
    def __getitem__(self, key):
        return getattr(self, key)

    def __repr__(self):
        return f'({self.items})'

    def is_positive(self):
        return all(map(lambda x: x>=0, self.items))
    
class State(object):
    """ a position, and setup within the game board and history """
    def __init__(self, play_history, inventory):
        self.inventory = inventory
        self.play_history = play_history
        
    def __hash__(self):
        return hash(str(self.play_history))

    def __repr__(self):
        return f'State({self.inventory}, history={self.play_history})'
    
    
    
#
# example Game is stateless
#
class Game(object):
    def __init__(self, inventory=None):
        self.initial_inventory = inventory if inventory else ItemSet([0,0])
    
    MOVES = [ItemSet([2,0]), ItemSet([-1,1])]
    
    def start(self):
        return State([], self.initial_inventory)
    
    def available_moves(self, state):
        return [m for m in Game.MOVES if (state.inventory + m).is_positive()]
    
    def next_state(self, state, move):
        #return new_state
        new_history = state.play_history.copy()
        new_history.append(move)
        
        return State(new_history, state.inventory + move) 
    
    def winner(self, state):
        """ return true if self.state is a winning state """
        #return winner
        if state.inventory.items[1] > 2:
            return True
        
        return False
    
    def __repr__(self):
        return f'Game(init={self.initial_inventory}, moves={self.MOVES})'
    
    
class MonteCarloNode(object):
    def __init__(self, parent, move, state, unexpanded_plays):
        self.move = move
        self.state = state
        
        self.n_plays = 0
        self.n_wins = 0
    
        self.parent = parent
        self.children = {
            m: {'play': m, 'node': None}
            for m in unexpanded_plays
        }
            
            
    def child_node(self, move):
        child = self.children[move]
        return child['node']
        
    def expand(self, move, child_state, unexpanded_plays):
        child_node = MonteCarloNode(self, move, child_state, unexpanded_plays)
        self.children[move] = {'play': move, 'node': child_node}
        return child_node
    
    def all_plays(self):
        return [child['play'] for _, child in self.children.items()]
    
    def unexpanded_plays(self):
        return [child['play'] for _, child in self.children.items() if child['node'] is None]
    
    def is_fully_expanded(self):
        any([child['node'] is None for _, child in self.children.items()])
        
    def is_leaf(self):
        return bool(self.children)
    
    def get_ucb1(self, bias):
        return self.n_wins / self.n_plays + Math.sqrt(bias * Math.log(self.parent.n_plays) / self.n_plays)
    
    def __repr__(self):
        return f'Node(move={self.move}, state={self.state})'
    
# utility method used in monte carlo object
#returns a random element of the list x
_pick_random = lambda x: x[random.randint(0, len(x))]
    
    
    
class MonteCarlo(object):
    """ https://github.com/quasimik/medium-mcts/blob/master/monte-carlo.js """
    def __init__(self, game, bias_param=2):
        self.game = game
        self.bias = bias_param
        self.nodes = {} # state->MonteCarloNode
        
    def run_search(self, state, timeout=1):
        self._make_node(state)
        
        draws, total_sims = 0, 0
        end = datetime.utcnow().timestamp() + timeout
        while (datetime.utcnow().timestamp() < end):
            node = self._select(state)
            print(f'node-search: {node}')
            winner = game.winner(node.state)
            
            if not winner and not node.is_leaf():
                node = self._expand(node)
                winner = self._simulate(node)
                
            self._backpropagate(node, winner)
            
            if not winner:
                draws += 1
                
            total_sims += 1
            
        return {
            'runtime': timeout,
            'simulations': total_sims,
            'draws': draws
        }
    
    
    def best_play(self, state, policy='robust'):
        self._make_node(state)
        node = self.nodes[state]
        all_plays = node.all_plays()
        
        if policy == 'robust':
            local_max = -math.inf
            for move in all_plays:
                child_node = node.child_node(move)
                if child_node and child_node.n_plays > local_max:
                    best_play = play
                    local_max = child_node.n_plays
        
        elif policy == 'max':
            local_max = -math.inf
            for move in all_plays:
                child_node = node.child_node(move)
                ratio = child_node.n_wins / child_node.n_plays
                if (ratio > local_max):
                    best_play = move
                    local_max = ratio
                    
        return best_play
    
    
    def _make_node(self, state):
        if state not in self.nodes:
            unexpanded_plays = self.game.available_moves(state).copy()
            self.nodes[state] = MonteCarloNode(None, None, state, unexpanded_plays)

            
    ## MCTS - phase1 - selection
    def _select(self, state):
        node = self.nodes[state]
        while node.is_fully_expanded() and not node.is_leaf():
            #plays = node.app_plays()
            best_ucb1 = -math.inf
            best_play = None
            for move in node.all_plays():
                child_ucb1 = node.child_node(move).get_ucb1(self.bias)
                if child_ucb1 > best_ucb1:
                    best_play = move
                    best_ucb1 = child_ucb1
            node = node.child_node(best_play)
        return node
                    
    # MCTS - phase2
    def _expand(self, node):
        """ pick a random number """
        unexpanded_moves = node.unexpanded_plays()
        move = _pick_random(unexpanded_moves)
        
        child_state = self.game.next_state(node.state, move)
        child_unexpanded_plays = self.game.available_moves(node.state)
        child_node = node.expand(move, child_state, child_unexpanded_plays)
        self.nodes[child_state] = child_node
        return child_node
    
    # MCTS - phase 3 - simulation
    def _simulate(self, node):
        state = node.state
        winner = self.game.winner(state)
        while winner is None:
            plays = game.available_moves(state)
            move = _pick_random(plays)
            state = self.game.next_state(state, move)
            winner = self.game.winner(state)
        return winner
    
    # MCTS = phase 4 - backpropagation
    def _backpropagate(self, node, winner):
        while node:
            node.n_plays += 1
            if winner: # TODO this maybe different for N-player games
                node.n_wins += 1
            node = node.parent
                
                
    # utility methods
    def get_stats(self, state):
        node = self.nodes[state]
        stats = {
            'n_plays': node.n_plays,
            'n_wins': node.n_wins,
            'children': []
        }
        for _, child in node.children.items():
            if child.node is None:
                stats['children'].append({'play': child['play'], 'n_plays': None, 'n_wins': None})
            else:
                stats['children'].append({'play': child['play'], 'n_plays': child.node['n_plays'], 'n_wins': child.node['n_wins']})
    
        return stats
    

In [3]:
#game = Game(ItemSet([1,0]))
#game.available_moves()

In [4]:
#game = Game()
game = Game()
mcts = MonteCarlo(game)

state = game.start()
winner = game.winner(state)
print(f'winner: {winner}')
# while winner is None:
#     mtcs.run_search(state, 1)
#     play = mtcs.best_play(state)
    
#     state = game.next_state(state, play)

winner: False


In [5]:
search_results = mcts.run_search(state, 2)

node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State

node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State

node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State

node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State

node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State

...

node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State

node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State

node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State

node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State

node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State(([0, 0]), history=[]))
node-search: Node(move=None, state=State

In [6]:
print(search_results)

{'runtime': 2, 'simulations': 24955, 'draws': 24955}


In [7]:
play = mcts.best_play(state)
print(f'best play: {play}')

UnboundLocalError: local variable 'best_play' referenced before assignment

In [ ]:
mcts.nodes